In [1]:
from infrastructure.auto_regressive_text_batch import AutoRegressiveTextBatch
from core.conversation import Conversation
from core.message import Message
from infrastructure.auto_regressive_text_model import AutoRegressiveTextModel
import uuid
from infrastructure.padding_side import PaddingSide

In [2]:
model_path ="assets\\Qwen2_0.5B_Instruct"

In [3]:
conversation1 = Conversation(messages=[

    Message(role='system', content="You are a helpful assistant."),
    Message(role='user', content="Give me a short introduction to large language model."),
    
])

In [4]:
conversation2 = Conversation(messages=[
    Message(role='user', content="Give me name for my small dog that has blue eyes and his color is white"),  
])

In [5]:
model = AutoRegressiveTextModel(model_path=model_path,padding_side=PaddingSide.left)
model.load_model()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
print(model.padding_side.value)

left


In [7]:
batch_input = AutoRegressiveTextBatch(conversations_list=[conversation1, conversation2],
                              attention_mask_list=[None, None],
                              input_ids_list=[None, None],
                              past_key_values=[None, None],
                              embedding_list=[None,None],
                              ids=[str(uuid.uuid4()), str(uuid.uuid4())],
                              is_done_list=[False, False],
                              next_tokens_ids_list=[None, None],
                              next_tokens_list=[None, None],)

In [19]:
processed_input_batch = model.process_batch_input(
    auto_regressive_batch=batch_input
)

In [20]:
processed_input_batch.attention_mask_list[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1])

In [21]:
processed_input_batch.input_ids_list[0]

tensor([151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
        151645,    198, 151644,    872,    198,  35127,    752,    264,   2805,
         16800,    311,   3460,   4128,   1614,     13, 151645,    198, 151644,
         77091,    198])

In [22]:
removed_batch_padding = model.remove_batch_padding(
    auto_regressive_batch=processed_input_batch
)

In [23]:
removed_batch_padding.input_ids_list[0]

tensor([151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
        151645,    198, 151644,    872,    198,  35127,    752,    264,   2805,
         16800,    311,   3460,   4128,   1614,     13, 151645,    198, 151644,
         77091,    198])

In [24]:
padded_batch = model.apply_batch_padding(
    auto_regressive_batch=removed_batch_padding
)

In [25]:
model_output = model.forward(auto_regressive_batch=padded_batch)

In [26]:
processed_model_output = model.remove_batch_padding(auto_regressive_batch=model_output)

In [27]:
processed_model_output.next_tokens_list

['A', 'Your']

In [28]:
output = model(batch_input)

In [29]:
output.next_tokens_list

['A', 'Your']